# Overview

**Note: All the images are from the Credits section or the internet. And this one was not reviewed completely because no enough computing resources, and bf16 type is not supported by current env.**

Pre-trained LLMs can only perform next-token prediction, making them unable to answer questions. This is why these base models are then fine-tuned on pairs of instructions and anwers to act as helpful assistants. However, this process can still be flawed: fine-tuned LLMs can be biased, toxic, harmful, etc. This is where Reinforcement Learning from RLHF(Reinforcement Learning from Human Feedback) comes into play.

RLHF provides different answers to the LLM, which are ranked according to a desired behavior(helpfulness, toxicity, etc). The model learns to output the best answer among these candidates, hence mimicking the behavior we want to instill. Often seen as a way to censor models, this process has recently become popular for improving performance, as shown in [neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1).

We are going to fine-tune OpenHermes-2.5 using a RLHF-like technique: **Direct Preference Optimization(DPO)**.

In [1]:
%%capture
!pip install transformers==4.36.2
!pip install datasets==2.15.0
!pip install peft==0.7.1
!pip install bitsandbytes==0.41.3
!pip install trl==0.7.7

In [2]:
import os
from huggingface_hub import login
#from kaggle_secrets import UserSecretsClient

#user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

# os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
# os.environ["WANDB_PROJECT"] = "Fine-tune-models"
# os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
# os.environ["WANDB_NAME"] = "ft-openhermes-25-mistral-7b-irca-dpo-pairs"

/mnt/host/cs234_final/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'user_secrets' is not defined

# Preparing datasets

We are prefer the **Preference datasets**. Typically consist of a collection of answers that are ranked by humans. This ranking is seential, as the RLHF process fine-tunes LLMs to output the preferred answer.

In [3]:
from datasets import load_dataset

train_ds=load_dataset("Intel/orca_dpo_pairs")["train"]

Generating train split: 100%|██████████| 12859/12859 [00:00<00:00, 190229.24 examples/s]


The structure of the dataset is straightforward: for each row, there is one chosen(preferred) answer, and one rejected answer. The goal of RLHF is to guide the model to output the preferred answer.

## Preference datasets

They are notoriously costly and difficult to make, as they require collecting manual feedback from humans. This feedback is also subjective and can easily be biased toward confident(but wrong) answers or contradict itself (different annotators have different values). Over time, several solutions have been proposed to tackle these issues, such as replacing human feedback with AI feedback [RLAIF](https://arxiv.org/abs/2212.08073).

These datasets also tend to be a lot smaller than fine-tuninf datasets. To illustrate this, the excellent [neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1) uses 518k samples for fine-tuning [Open-Orca/SlimOrca](https://huggingface.co/datasets/Open-Orca/SlimOrca) but only 12.9k samples for RLHF [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs). In this case, the authors generated answers with GPT-4/3.5 to create the preferred answers, and with [Llama 2 13b chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) to create the rejected responses. It's a smart way to bypass human feedback and only rely on models with different levels of performance.

# Direct Preference Optimization

While the concept of RLHF has been used in robotics for a long time, it was popularized for LLMs in [Fine-tuning Language Models from Human Preferences](https://arxiv.org/pdf/1909.08593.pdf). In this paper, the authors present a framework where a reward model is trained to approximate human feedback. This reward model is then used to optimize the fine-tuned model's policy using the [Proximal Policy Optimization algorithm](https://arxiv.org/abs/1707.06347).

<div style="text-align: center"><img src="https://files.mastodon.social/media_attachments/files/111/707/466/912/510/214/original/5f3fa6b5a0b186a3.webp" width="60%" heigh="60%" alt="proximal policy optimization"></div>


The Core concept of PPO revolves around making smaller, incremental updates to the policy, as larger updates can lead to instability or suboptimal solutions. From experience, this technique is unfortunately still unstable(loss diverges), difficult to reproduce(numerous hyperparameters, sensitive to random seeds), and computationally expensive.

This is where Direct Preference Optimization(DPO) comes into play. DPO simplifies control by treating the task as a classification problem. Concretely, it uses two models: the **trained model(or policy model)** and a copy of it called the **reference model**. During training, the goal is to make sure the trained model outputs higher probabilities for preferred answers than the reference model. Conversely, we also want it output lower probabilities for rejected answers. It means we're penalizing the LLM for bad answers and rewarding it for good ones.

<div style="text-align: center"><img src="https://files.mastodon.social/media_attachments/files/111/707/528/109/748/161/original/7c1ef620f48dd4a6.webp" width="60%" heigh="60%" alt="direct preference optimization"></div>

By using the LLM itself as a reward model and employing binary cross-entropy objectives, DPO efficiently aligns the model's outputs with human preferences without the need for extensive sampling, reward model fitting, or intricate hyperparameters adjustments. It results in a more stable, more efficient, and computationally less demanding process.

# Formatting the data

Here we are going to fine-tune [OpenHermes-2.5-Mistral-7B](https://huggingface.co/teknium/OpenHermes-2.5-Mistral-7B) which is a Mistral-7b model that was only supervised fine-tuned. To this end, we will use the [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs) dataset to align our model and improve this performance.

In [6]:
#!pip install -U peft
!pip install git+https://github.com/huggingface/peft
!pip list | grep peft

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-_4dcbg8u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-_4dcbg8u
  Resolved https://github.com/huggingface/peft to commit 03798a9143c90d796a0bee8f43863668d084381f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.11.2.dev0-py3-none-any.whl size=255725 sha256=0a720b3850cbff30d1e38d6d5e343a615792d2d1f6824b4a5af9e37d0db39c2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-m87s43r5/wheels/4c/16/67/1002a2d4daa822eff130e6d85b90051b75d2ce0d26b9448e4a
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.11.1
    Uninstalling peft-0.11.1:
      Successfully uninstalled peft-0.11.1
peft                      0.11.2.dev0


In [7]:
from peft import LoraConfig, PeftModel, get_peft_model
from trl import DPOTrainer

ModuleNotFoundError: No module named 'peft'

In [ ]:
model_name="teknium/OpenHermes-2.5-Mistral-7B"

OpenHermes-2.5-Mistral-7B uses a specific chat template, called [ChatML](https://huggingface.co/docs/transformers/chat_templating). Here is an example of conversation formatted with this template:

```
<|im_start|>system
You are a helpful chatbot that will do its best not to say anything so stupid that people tweet about it.<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I'm doing great!<|im_end|>
```

Each of sentence which includes a role, like "system, "user" or "assistant", and it appends special tokens at the beginning and the end of the sentence(<|im_start|> and <|im_end|>). These two are used to separate different sentences. Moreover, DPOTrainer also requires a specific format with three columns: prompt, chosen and rejected.

We will simply concatenate the system and question columns to the prompt column. We will also map the chatgpt column to "chosen" and llama2-13b-chat to "rejected". To format the dataset in a reliable way, we will use the tokenizer's `apply_chat_template()` function, which already use ChatML.

In [ ]:
from transformers import AutoTokenizer

def chatml_format(example):
    # format system
    if len(example['system'])>0:
        message ={"role":"system","content":example['system']}
        system=tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system=""
    
    #format instruction
    message={"role":"user","content":example['question']}
    prompt=tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    
    # format chosen answer
    chosen =example['chosen']+"<im_end>\n"
    
    # format rejected answer
    rejected = example['rejected']+"<im_end>\n"
    
    return {
        "prompt": system+prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# we have load datasets in above section

# save columns
original_columns=train_ds.column_names

# tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"

#format dataset
train_dataset=train_ds.map(
    function=chatml_format,
    remove_columns=original_columns
)

# checking only one example
train_dataset[0]

# Training the model with DPO

We define the LoRA configurations to train the model. We set the rank value to be equal to the `lora_alpha`, which is unusual(2*`r` as a rule of thumb). We also target all the linear modules with adapters.

In [ ]:
from transformers import BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model

import torch

# LoRA configuration
peft_config=LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj','gate_proj','v_proj','up_proj','q_proj','o_proj','down_proj']
)

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
#     llm_int8_enable_fp32_cpu_offload=True,
)

## Quantize the model

We're now ready to load the model we want to fine-tune with DPO. In this case, two models are required: the model to fine-tune as well as the reference model. This is mostly for the sake of readability, as the `DPOTrainer` object automatically creates a ference model if none is provided.

In [ ]:
from transformers import AutoModelForCausalLM

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)

model

In [ ]:
model.config.use_cache=False
model=get_peft_model(model, peft_config)
model.get_memory_footprint()
model.config

In [ ]:
# Reference model
ref_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
ref_model=get_peft_model(ref_model, peft_config)
ref_model.get_memory_footprint()
ref_model.config

The final step consists of providing all the hyperparameters to `TrainingArguments` and `DPOTrainer`:
* Among them, the `beta` parameter is unique to DPO since it controls the divergence from the initial policy(0.1 is a typical value for it).

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer

training_args=TrainingArguments(
    per_device_train_batch_size=2,
    # key word argumnts to be passed to the gradient_checkingpointing_enable method
    gradient_checkpointing=True,
    gradient_accumulation_steps=5,
    remove_unused_columns=False,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    # the total number of training steps to perform
    max_steps=100,
    save_strategy="no",
    logging_steps=100,
    output_dir=os.getenv("WANDB_NAME"),
    optim="paged_adamw_32bit",
    bf16=False, # Doesn;t support in Kaggle environment
    fp16=True,
    # Number of steps used for a linear warmup from 0 to learning_rate
    warmup_steps=50,
    run_name=os.getenv("WANDB_NAME"),
    report_to="wandb"
)

dpo_trainer=DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Kaggle env does not have enough Memory to doing training, it will restarted and cause error.
dpo_trainer.train()

# Saving the merged model

In [ ]:
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16(instead of NF4)
base_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_type=torch.float16,
)

tokenizer=AutoTokenizer.from_pretrained(model_name)

# merge base model with the adapter
model=PeftModel.from_pretrained(base_model, "final_checkpoint")
model=model.merge_and_unload()

# save model and tokenizer
model.save_pretrained(os.getenv("WANDB_NAME"))
tokenizer.save_pretrained(os.getenv("WANDB_NAME"))


model.push_to_hub(os.getenv("WANDB_NAME"))
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))

# Checking the model performs

Excepting run the model locally, we can also leverage the [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) to evaluate it. As the process is quite resource-intensive, we can also directly submit it for evaluation on the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

In [ ]:
# format prompt
message=[
    {"role":"system", "content":"The weather is important."},
    {"role":"user","content":"Does Melborune raining today?"}
]

tokenizer=AutoTokenizer.from_pretrained(os.getenv("WANDB_NAME"))
prompt=tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

pipe=transformers.pipeline(
    "text-generation",
    model=os.getenv("WANDB_NAME"),
    tokenizer=tokenizer
)

# generate text
sequences=pipe(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)

sequences[0]['generated_text']

# Credits

* https://towardsdatascience.com/fine-tune-a-mistral-7b-model-with-direct-preference-optimization-708042745aac